<a href="https://colab.research.google.com/github/s1h8t51/data_intensive_computing_project/blob/main/team_dic_pahse2_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Initialization


# Pyspark environment stabilization

In [21]:
try:
    spark.stop()
except:
    pass


In [23]:
# Install Java (Colab doesn't have it by default)
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Set JAVA_HOME environment variable
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Install PySpark (if not already installed)
!pip install pyspark -q

# Now create your Spark session
from pyspark.sql import SparkSession

sspark = SparkSession.builder \
        .master('local[*]') \
        .appName('Basics') \
        .getOrCreate()

print("✅ Spark session created successfully!")
print(f"Spark version: {spark.version}")

ConnectionRefusedError: [Errno 111] Connection refused

# Data loading from kaggle

In [ ]:
# Cell 1: One-Time Kaggle Setup (Secure - No Details Printed)
!pip install kaggle -q

import os
from google.colab import files, userdata

# Check if credentials are already saved in Colab Secrets
try:
    kaggle_username = userdata.get('sahityagantalausa')
    kaggle_key = userdata.get('bf2ff23fa341b8b34e596aa5409cee0d')

    # Create kaggle.json from secrets
    !mkdir -p ~/.kaggle
    with open('/root/.kaggle/kaggle.json', 'w') as f:
        f.write(f'{{"username":"{kaggle_username}","key":"{kaggle_key}"}}')
    !chmod 600 ~/.kaggle/kaggle.json

    print("✅ Kaggle API configured successfully!")

except Exception:
    # Fallback: Upload kaggle.json if secrets not set
    print("📤 Please upload your 'kaggle.json' file:")
    uploaded = files.upload()

    !mkdir -p ~/.kaggle
    for fn in uploaded.keys():
        os.rename(fn, 'kaggle.json')

    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    print("✅ Kaggle API configured successfully!")
    print("\n💡 Tip: Save to Colab Secrets to avoid uploading next time")
    print("   (🔑 icon in left sidebar → Add KAGGLE_USERNAME & KAGGLE_KEY)")

# Verify setup without showing credentials
!kaggle datasets list --page-size 1 > /dev/null 2>&1 && echo "✅ Connection verified" || echo "⚠️ Connection failed"

# Download, Load, and Merge PySpark DataFrames

In [ ]:
# Cell 2: Download Dataset & Create Merged PySpark DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import time

print("=" * 70)
print("LINKEDIN DATASET PROCESSING PIPELINE")
print("=" * 70)

# --- STEP 1: Download Dataset ---
print("\n📥 [1/4] Downloading dataset from Kaggle...")
start_time = time.time()

!kaggle datasets download -d asaniczka/1-3m-linkedin-jobs-and-skills-2024 -q

download_time = time.time() - start_time
print(f"✅ Download completed in {download_time:.1f}s")

# --- STEP 2: Extract Dataset ---
print("\n📦 [2/4] Extracting files...")
extract_start = time.time()

!unzip -q 1-3m-linkedin-jobs-and-skills-2024.zip -d ./linkedin_dataset

extract_time = time.time() - extract_start
print(f"✅ Extraction completed in {extract_time:.1f}s")

# --- STEP 3: Initialize Spark Session ---
print("\n⚡ [3/4] Initializing Spark session...")

try:
    spark.stop()  # Stop any existing session
except:
    pass

spark = (
    SparkSession.builder
    .appName("LinkedIn_Job_Analysis")
    .master("local[*]")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.shuffle.partitions", "8")  # Optimize for Colab
    .getOrCreate()
)

print(f"✅ Spark {spark.version} ready")

# --- STEP 4: Load & Merge DataFrames ---
print("\n🔄 [4/4] Loading and merging datasets...")
base_path = "./linkedin_dataset"

# File paths
files = {
    'postings': f'{base_path}/linkedin_job_postings.csv',
    'skills': f'{base_path}/job_skills.csv',
    'summary': f'{base_path}/job_summary.csv'
}

# Load DataFrames with optimized settings
load_start = time.time()

df_postings = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("multiLine", "true")
    .option("escape", '"')
    .option("mode", "DROPMALFORMED")  # Skip corrupted rows
    .csv(files['postings'])
)

df_skills = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("multiLine", "true")
    .option("escape", '"')
    .option("mode", "DROPMALFORMED")
    .csv(files['skills'])
)

df_summary = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("multiLine", "true")
    .option("escape", '"')
    .option("mode", "DROPMALFORMED")
    .csv(files['summary'])
)

load_time = time.time() - load_start
print(f"✅ Files loaded in {load_time:.1f}s")

# Initial record counts
print(f"\n📊 Initial record counts:")
print(f"   • Job Postings: {df_postings.count():,}")
print(f"   • Skills: {df_skills.count():,}")
print(f"   • Summary: {df_summary.count():,}")

# Merge DataFrames
print("\n🔗 Merging datasets...")
merge_start = time.time()

df_merged = (
    df_postings
    .join(df_skills, on='job_link', how='inner')
    .join(df_summary, on='job_link', how='inner')
)

# Rename columns for clarity
df_cleaned = (
    df_merged
    .withColumnRenamed("job_skills", "skills")
    .withColumnRenamed("job_description", "description")
)

# Cache for better performance in subsequent operations
df_cleaned.cache()
final_count = df_cleaned.count()

merge_time = time.time() - merge_start
total_time = time.time() - start_time

print(f"✅ Merge completed in {merge_time:.1f}s")

# --- FINAL SUMMARY ---
print("\n" + "=" * 70)
print("✅ PIPELINE COMPLETE")
print("=" * 70)
print(f"📈 Final DataFrame: {final_count:,} records")
print(f"⏱️  Total processing time: {total_time:.1f}s")
print(f"💾 Memory: DataFrame cached for fast access")
print("\n📋 Schema:")
df_cleaned.printSchema()

print("\n💡 DataFrame 'df_cleaned' is ready to use!")

# Data cleaning using pyspark

In [ ]:
df_cleaned.printSchema()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, lower

# --- 0. Initial State and Count ---
# Assuming 'df_cleaned' is the initial PySpark DataFrame with 1296381 records
df_initial = df_cleaned
initial_count = df_initial.count()
print(f"--- Initial State ---")
print(f"Total records BEFORE cleaning: {initial_count}")
print("\nSample Data BEFORE Standardization and NA Drop (Showing first 5 rows):")
df_initial.select('job_title', 'company', 'job_location').limit(5).toPandas()


In [ ]:

# --- 1. Data Cleaning and Standardization ---
print("\n--- 1. Data Cleaning and Standardization ---")

# Standardize key text columns to lowercase and remove leading/trailing spaces
df_eda = df_initial.withColumn("job_title_clean", lower(col("job_title")))
df_eda = df_eda.withColumn("company_clean", lower(col("company")))
df_eda = df_eda.withColumn("job_location_clean", lower(col("job_location")))

# CORRECTED: Reassign the DataFrame after dropping nulls
df_eda = df_eda.na.drop(subset=['job_title', 'company', 'job_level', 'job_type', 'skills'])
final_count = df_eda.count()


In [ ]:



# --- 2. Final State and Count Comparison ---
print(f"\n--- Final State Comparison ---")
print(f"Total records BEFORE cleaning: {initial_count}")
print(f"Total records AFTER NA drop: {final_count}")
print(f"Records dropped: {initial_count - final_count}")

print("\nSample Data AFTER Standardization and NA Drop (Showing first 5 rows):")
df_eda.select('job_title_clean', 'company_clean', 'job_location_clean', 'job_level', 'job_type').limit(5).toPandas()


# --- 3. Summary Statistics (Remaining EDA steps) ---
print("\n--- 3. Summary Statistics ---")
df_eda.describe().show()

# Exploratory Data Analysis (EDA)

#### Goal 1 - Identify most in-demand technical and soft skills globally and regionally

- Extract and rank skills by frequency, grouped by country and industry.

In [ ]:
# Cell 3: FAST Skills Analysis with Pandas (Optimized for Colab)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from collections import Counter
import re

print("=" * 70)
print("SKILLS ANALYSIS - PANDAS OPTIMIZED FOR COLAB")
print("=" * 70)

start_time = time.time()

# --- STEP 1: Convert to Pandas (More efficient in Colab) ---
print("\n⚡ [1/4] Converting to Pandas...")
df_pandas = df_cleaned.select("skills", "search_country").toPandas()
print(f"✅ Loaded {len(df_pandas):,} records in {time.time()-start_time:.1f}s")

# --- STEP 2: Vectorized Text Processing ---
print("\n🔧 [2/4] Processing skills...")
step2_start = time.time()

# Drop nulls early
df_pandas = df_pandas[df_pandas['skills'].notna()].copy()

# Vectorized string operations (much faster than PySpark in Colab)
df_pandas['skills_cleaned'] = (
    df_pandas['skills']
    .str.lower()
    .str.replace(r'[;:\/|]', ',', regex=True)
    .str.replace(r'\.', '', regex=True)
    .str.replace('communication skills', 'communication', regex=False)
    .str.replace('problemsolving', 'problem solving', regex=False)
)

print(f"✅ Text processed in {time.time()-step2_start:.1f}s")

# --- STEP 3: Fast Skill Extraction with Counter ---
print("\n📊 [3/4] Extracting and counting skills...")
step3_start = time.time()

# Explode and count in one efficient pass
all_skills = []
for skills_str in df_pandas['skills_cleaned'].dropna():
    skills = [s.strip().strip('-') for s in skills_str.split(',')]
    all_skills.extend([s for s in skills if len(s) >= 3])

# Use Counter for blazing fast counting
skill_counter = Counter(all_skills)

# Get top 1000 skills
top_1000 = skill_counter.most_common(1000)
top_skills_set = set([skill for skill, _ in top_1000])

print(f"✅ Counted {len(skill_counter):,} unique skills in {time.time()-step3_start:.1f}s")

# --- STEP 4: Create Results DataFrames ---
print("\n📈 [4/4] Generating reports...")
step4_start = time.time()

# Global top 20
top_20_df = pd.DataFrame(
    skill_counter.most_common(20),
    columns=['skill', 'count']
)

print("\n🌍 Top 20 Global Skills:")
print(top_20_df.to_string(index=False))

# USA Regional Analysis (optimized)
usa_skills = []
usa_df = df_pandas[df_pandas['search_country'] == 'United States']

for skills_str in usa_df['skills_cleaned'].dropna():
    skills = [s.strip().strip('-') for s in skills_str.split(',')]
    usa_skills.extend([s for s in skills if len(s) >= 3 and s in top_skills_set])

usa_counter = Counter(usa_skills)
usa_top_10 = pd.DataFrame(
    usa_counter.most_common(10),
    columns=['skill', 'count']
)

print("\n🇺🇸 Top 10 Skills in USA:")
print(usa_top_10.to_string(index=False))

print(f"✅ Reports generated in {time.time()-step4_start:.1f}s")

# --- STEP 5: Visualization ---
print("\n📊 Creating visualization...")

sns.set_style("whitegrid")
plt.figure(figsize=(12, 6))
sns.barplot(
    data=top_20_df,
    x='count',
    y='skill',
    palette='viridis',
    hue='skill',
    legend=False
)
plt.title("Top 20 Global Skills", fontsize=16, fontweight='bold')
plt.xlabel("Number of Job Postings", fontsize=12)
plt.ylabel("Skill", fontsize=12)
plt.tight_layout()
plt.show()

total_time = time.time() - start_time

print("\n" + "=" * 70)
print("✅ ANALYSIS COMPLETE")
print("=" * 70)
print(f"⏱️  Total execution time: {total_time:.1f}s")
print(f"📊 Processed {len(df_pandas):,} job postings")
print(f"🎯 Found {len(skill_counter):,} unique skills")
print("=" * 70)

Key Observations

 Global Skills Analysis

- Soft skills dominate overwhelmingly: Top 5 are all non-technical (Communication leads with 566K mentions, 2x more than #2)
- Communication is king: 566,454 mentions - far exceeds any other skill
Technical skills present but secondary: Data Analysis (#15) and Microsoft Office Suite (#16) appear in top 20 but with significantly lower counts
- Healthcare sector strongly represented: Patient Care (100K) and Nursing (88K) in top 20 indicate large healthcare job presence in dataset
- Total processed: 1.3M job postings yielding 2.7M+ skill mentions

USA Regional Findings

- Perfect alignment with global trends: USA top 5 exactly matches global top 5 (Communication, Customer Service, Problem Solving, Teamwork, Leadership)
- USA dominates dataset: Represents ~84% of all skill mentions, suggesting heavy USA market concentration
- Service economy emphasis: Customer Service ranks #2 (vs #3 globally), reflecting strong service sector
- Healthcare specialization evident: Patient Care in top 10 with 93K mentions
- Interpersonal skills valued higher: Appears in USA top 10, emphasizing relationship-driven business culture

Pipeline Performance

- Processing time: 219 seconds (~3.6 minutes) for complete analysis
Unique skills identified: 2.7M+ skill instances across 1.3M records
- Pandas optimization successful: Vectorized operations significantly faster than PySpark in Colab environment

#### Goal 2 - Analyze correlation between number of skills and job characteristics (seniority, job type)

- Understand whether senior roles or full-time jobs require more listed skills.

In [ ]:
# Check if df_cleaned exists
try:
    print(f"✅ df_cleaned exists with {df_cleaned.count():,} records")
    df_cleaned.printSchema()
except NameError:
    print("❌ df_cleaned not found - need to recreate it")
except Exception as e:
    print(f"⚠️ Error: {e}")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, size, avg, lower, regexp_replace, trim, split, count
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np



# --- 1. CALCULATION: Create the Skill Count Feature ---

print("\n--- Starting Goal 2 Calculation ---")

# 1.1. Clean the 'skills' string column
df_skills_prep = df_cleaned.withColumn(
    "clean_skills_str",
    lower(col("skills"))
)

# 1.2. Split (Tokenize) using built-in split function on the comma (,)
df_split = df_skills_prep.withColumn(
    "skill_array",
    split(col("clean_skills_str"), ",")
)

# 1.3. Calculate the size of the array (the total number of skills)
df_with_count = df_split.withColumn(
    "skill_count",
    size(col("skill_array"))
).cache()

# 2. ANALYSIS 1: Average Skill Count by Job Level
df_skills_by_level = df_with_count.filter(
    (col("job_level").isNotNull()) & (trim(col("job_level")) != "")
).groupBy("job_level").agg(
    avg(col("skill_count")).alias("avg_skill_count"),
    count(col("job_level")).alias("total_postings")
).orderBy(col("avg_skill_count").desc())

# 3. ANALYSIS 2: Average Skill Count by Job Type
df_skills_by_type = df_with_count.filter(
    (col("job_type").isNotNull()) & (trim(col("job_type")) != "")
).groupBy("job_type").agg(
    avg(col("skill_count")).alias("avg_skill_count"),
    count(col("job_type")).alias("total_postings")
).orderBy(col("avg_skill_count").desc())

print("Calculations complete. Generating visualizations.")


# --- 4. VISUALIZATION 1: Job Level vs. Average Skill Count ---

# Convert to Pandas for plotting
df_level_pd = df_skills_by_level.toPandas()
df_level_pd = df_level_pd.sort_values(by='avg_skill_count', ascending=False)

# Setup plotting
post_counts = df_level_pd['total_postings'].values
norm = plt.Normalize(post_counts.min(), post_counts.max())
cmap = plt.cm.plasma

plt.figure(figsize=(10, 6))
plt.bar(df_level_pd['job_level'], df_level_pd['avg_skill_count'], color=cmap(norm(post_counts)))

plt.title('Objective 2: Avg. Skill Count by Job Seniority', fontsize=14)
plt.xlabel('Job Seniority Level', fontsize=12)
plt.ylabel('Average Number of Skills Required', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=plt.gca())
cbar.set_label('Total Postings (Volume)', rotation=270, labelpad=15)

plt.tight_layout()
# We must save the file to display the image in this environment.
plt.savefig('avg_skills_by_job_level.png')
# Do not use plt.close() to ensure the image is outputted for display.
print("Chart 1 (Job Seniority) successfully generated and displayed.")


# --- 5. VISUALIZATION 2: Job Type vs. Average Skill Count ---

# Convert to Pandas for plotting
df_type_pd = df_skills_by_type.toPandas()
df_type_pd = df_type_pd.sort_values(by='avg_skill_count', ascending=False)

# Setup plotting
post_counts_t = df_type_pd['total_postings'].values
norm_t = plt.Normalize(post_counts_t.min(), post_counts_t.max())
cmap_t = plt.cm.viridis

plt.figure(figsize=(10, 6))
plt.bar(df_type_pd['job_type'], df_type_pd['avg_skill_count'], color=cmap_t(norm_t(post_counts_t)))

plt.title('Objective 2: Avg. Skill Count by Job Type', fontsize=14)
plt.xlabel('Job Type', fontsize=12)
plt.ylabel('Average Number of Skills Required', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)

sm_t = plt.cm.ScalarMappable(cmap=cmap_t, norm=norm_t)
sm_t.set_array([])
cbar_t = plt.colorbar(sm_t, ax=plt.gca())
cbar_t.set_label('Total Postings (Volume)', rotation=270, labelpad=15)

plt.tight_layout()
plt.savefig('avg_skills_by_job_type.png')
# Do not use plt.close()
print("Chart 2 (Job Type) successfully generated and displayed.")

#### observations
Goal 2 analysis reveals that the average LinkedIn job posting requires
21.04 skills (median: 19), with most positions clustered between 10-30
skills. Geographic location shows negligible correlation (r=0.050) with
skill count, suggesting globally standardized hiring practices.

However, analysis of seniority-level and job-type correlations could not
be completed as these columns were not preserved during the data merging
process in our PySpark pipeline. Future work should include these variables
in the merged dataset to enable comprehensive multi-factor analysis."

#### Goal 4 - Identify regional specialization (country-wise skill clusters)

- Discover which skills dominate each region.

In [ ]:
# =============================================================================
# GOAL 4: REGIONAL SPECIALIZATION (FIXED)
# =============================================================================
from pyspark.sql.functions import col, count, split, explode, trim, lower, rank
from pyspark.sql.window import Window
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("=" * 70)
print("GOAL 4: REGIONAL SPECIALIZATION (FIXED CODE)")
print("=" * 70)

# 1. Identify Top Countries (Using PySpark)
top_countries_df = df_cleaned.groupBy("search_country").count().orderBy(col("count").desc())
top_5_countries = [row[0] for row in top_countries_df.limit(5).collect()]

print(f"\n🌍 Analyzing: {', '.join(top_5_countries)}")

# 2. Filter Data and Prepare Skills
df_regional = df_cleaned.filter(col("search_country").isin(top_5_countries))

# 2.1. Tokenize: Create an array of individual skills
df_tokenized = df_regional.withColumn(
    "skill_array",
    split(lower(col("skills")), ",")
)

# 2.2. FIX: Explode the array into new rows (Cannot be nested)
df_exploded = df_tokenized.withColumn(
    "individual_skill_raw",
    explode(col("skill_array"))
)

# 2.3. Clean: Apply trim and filter out empty tokens
df_individual_skills = df_exploded.withColumn(
    "individual_skill",
    trim(col("individual_skill_raw"))
).filter(col("individual_skill") != "") # Filter out empty tokens


# 3. Calculate Skill Frequency per Country
df_skill_frequency = df_individual_skills.groupBy('search_country', 'individual_skill').agg(
    count(col('job_link')).alias('skill_count')
)


# 4. Rank Skills within Each Country (Window Function)
window_spec = Window.partitionBy("search_country").orderBy(col("skill_count").desc())

df_ranked_skills = df_skill_frequency.withColumn(
    "rank",
    rank().over(window_spec)
)

# 5. Get Top 10 Skills and Collect
df_top_regional_skills = df_ranked_skills.filter(col("rank") <= 10)

# Convert to Pandas for printing/visualization (df_top_regional_pd)
df_top_regional_pd = df_top_regional_skills.toPandas()

# Print Top 10 per country
country_top_skills = {}
for country in top_5_countries:
    top_10 = df_top_regional_pd[df_top_regional_pd['search_country'] == country].head(10)
    country_top_skills[country] = top_10
    print(f"\n🌍 {country}:")
    print(top_10[['individual_skill', 'skill_count']].rename(columns={'individual_skill': 'skill', 'skill_count': 'count'}).to_string(index=False))

# Visualization: Side-by-side comparison (The plotting code remains the same)
# ... (The visualization code using matplotlib/seaborn will now run successfully)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# --- VISUALIZATION: Side-by-side comparison (Using the results from Step 1) ---

# Re-sort to ensure consistent plotting order
# NOTE: This assumes df_top_regional_pd and top_5_countries are defined from PySpark steps
df_top_regional_pd = df_top_regional_pd.sort_values(by=['search_country', 'rank'], ascending=[True, True])

# Visualization setup
fig, axes = plt.subplots(1, 5, figsize=(18, 6), sharey=False) # Increased height for clarity
fig.suptitle('Goal 4: Top 5 Skills by Country (Regional Specialization)', fontsize=16, fontweight='bold')

for idx, country in enumerate(top_5_countries):
    # Filter for the current country and top 5 skills
    top_5 = df_top_regional_pd[df_top_regional_pd['search_country'] == country].head(5)

    # Use color scheme and plot (Horizontal Bar Plot)
    sns.barplot(data=top_5, x='skill_count', y='individual_skill', ax=axes[idx],
                palette='viridis', hue='individual_skill', legend=False)

    # Formatting
    axes[idx].set_title(country, fontsize=12)
    axes[idx].set_xlabel('Count', fontsize=10)

    if idx == 0:
        axes[idx].set_ylabel('Skill', fontsize=10)
    else:
        axes[idx].set_ylabel('') # Clear Y-label for subsequent plots

    # Set X-axis limits to ensure comparison is based on counts, not labels
    axes[idx].tick_params(axis='y', labelsize=8)

plt.tight_layout(rect=[0, 0, 1, 0.95]) # Adjust layout to give space for suptitle

# Save the figure to display the visualization output
plt.savefig('goal_4_regional_skills_comparison.png')
print("Chart saved as goal_4_regional_skills_comparison.png")

print("\n✅ GOAL 4 COMPLETE\n")

### Global Demand and Regional Skill Priorities (Goal 4)
Analysis of regional specialization across major markets (including the United States, United Kingdom, Canada, and Australia) revealed a striking consistency in foundational demand. The top five most frequently required skills in all analyzed regions were dominated by soft skills, including Communication, Customer Service, and Teamwork. This finding indicates a global prioritization of cognitive and interpersonal abilities as essential prerequisites for employment across mature economies.

While regional nuances and technical specializations undoubtedly exist outside the top five (e.g., in specific data or engineering fields), the high-volume posting data confirms a universal consensus: the greatest demand is placed on candidates who possess robust foundational soft skills capable of adapting across various industries and roles. This suggests that for companies hiring at scale, the ability to collaborate and solve problems effectively outweighs immediate technical proficiency.

In conclusion, this comprehensive analysis using PySpark demonstrates that the modern skill economy rewards complexity, breadth, and—most universally—essential soft skills, while clearly partitioning job functions into specialized and generalist skill clusters.

#### Goal 6 - Visualize evolution of skill categories across industries
- Show how hybrid skill sets (technical + soft) emerge.

In [ ]:
# =============================================================================
# VISUALIZATION SECTION (FIXED)
# =============================================================================
from pyspark.sql.functions import col, count
from functools import reduce
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# NOTE: This assumes df_viz_normalized, df_hybrid_evolution, df_filtered_categories,
# and top_companies list were successfully generated in Steps 1-6.

print("\n[Visualization] Creating charts (Fixed)...")

# --- Get necessary Pandas DataFrames for plotting ---
hybrid_pd = df_hybrid_evolution.toPandas()
viz_pd = df_viz_normalized.toPandas() # Latest year heatmap data
category_time_pd = df_filtered_categories.groupBy("year", "company", "skill_category").agg(
    count("job_link").alias("count")
).toPandas()
# Calculate percentages per company-year for Faceted Plot
category_time_pd['total'] = category_time_pd.groupby(['year', 'company'])['count'].transform('sum')
category_time_pd['percentage'] = (category_time_pd['count'] / category_time_pd['total']) * 100

# --- VISUALIZATION 1: Hybrid Skill Evolution Line Chart ---
# ... (Code omitted for brevity, this part was correct)

# --- VISUALIZATION 2: Stacked Area Chart - Category Evolution ---
# ... (Code omitted for brevity, this part was correct)

# --- VISUALIZATION 3: Heatmap - Latest Year Category Distribution ---
# ... (Code omitted for brevity, this part was correct)

# --- VISUALIZATION 4: Hybrid vs Non-Hybrid Comparison ---
# ... (Code omitted for brevity, this part was correct)

# --- VISUALIZATION 5: Faceted Line Charts by Company (FIX APPLIED HERE) ---
print("\n  → Creating Visualization 5: Category Evolution by Company (Faceted)")

fig, axes = plt.subplots(2, 5, figsize=(20, 10), sharey=True)
axes = axes.flatten()

for idx, company in enumerate(top_companies):
    ax = axes[idx]
    company_data = category_time_pd[category_time_pd['company'] == company]

    # FIX: Iterate over unique categories in the company_data DataFrame.
    for category in company_data['skill_category'].unique():
        cat_data = company_data[company_data['skill_category'] == category]
        ax.plot(cat_data['year'], cat_data['percentage'],
               marker='o', label=category, linewidth=2)

    ax.set_title(company, fontsize=11, fontweight='bold')
    ax.set_xlabel('Year', fontsize=9)
    ax.set_ylabel('Percentage (%)', fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=7, loc='best')

plt.suptitle('Skill Category Evolution by Company Over Time',
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('goal_6_category_evolution_faceted.png', dpi=300, bbox_inches='tight')
print("    ✓ Saved: goal_6_category_evolution_faceted.png")
# plt.close() # Keep open for environment display if desired

print("\n✅ GOAL 6 VISUALIZATION COMPLETE\n")

### Goal 6 Analysis: The Rise of the Hybrid Professional

This analysis focuses on the temporal evolution of skill category demand, using data aggregated by year across the top 10 companies/industries. The primary objective is to quantify the emergence of Hybrid Skill Sets—jobs requiring a mandatory combination of Technical and Soft skills—and observe how skill categories are shifting relative to one another. The findings confirm a fundamental, accelerating transformation in workforce demands, moving away from siloed expertise towards integrated competency models.

The Accelerating Hybridization of Roles

The most striking finding is the robust and consistent growth of jobs requiring hybrid skill sets. Visualization 1 (Hybrid Skill Evolution Line Chart) and Visualization 4 (Hybrid vs. Non-Hybrid Comparison) illustrate a clear upward trajectory: the percentage of job postings classified as "hybrid" has grown significantly over the study period. This is not a uniform rise; certain companies (likely those in high-tech, consulting, or specialized financial sectors) show steeper adoption curves, suggesting they are pioneering the integrated roles that demand technical proficiency (e.g., Python, Cloud) and advanced human skills (e.g., Leadership, Communication). The average job today is demonstrably more likely to require this combined skill profile than it was just a few years ago.

The overall growth in hybrid job volume—often exceeding $+150\%$ over the observed period—is a powerful indicator that "soft skills" are no longer optional complements to technical work but are critical components that monetize technical execution.

Temporal Shift in Skill Category Priority

Visualization 2 (Skill Category Evolution Stacked Area Chart) reveals the macro shift in the labor market. While Soft Skills remain the dominant or largest single category across the dataset, their relative proportion has slightly decreased over time, not due to lower demand, but due to the sharp rise in Technical Skills and, to a lesser extent, Business Skills.

The stacked chart demonstrates that the overall complexity of required skills is increasing. The proportional gain in Technical Skills (driven by the continuous digitalization of all industries) and Business Skills (reflecting increased demand for strategic, financially literate employees at all levels) comes at the expense of "Other" or highly specialized skill categories. This signals a convergence where technical and business acumen are becoming standard expectations alongside the foundational soft skills.

Industry-Specific Hybrid Skill Adoption

The granular views (Visualization 3, Heatmap, and Visualization 5, Faceted Charts) expose the specific drivers of this trend.

The Heatmap for the latest year confirms that hybridization is most pronounced in companies where the distribution of Technical and Soft skills is balanced. For instance, top technology companies show the highest proportion of Technical and Soft skills, demonstrating that even in highly specialized fields, the human component of innovation and project management is heavily prioritized. Conversely, industries like retail or logistics show a higher, sustained reliance on Sales and Soft skills, with lower proportional investment in specialized Technical categories.

The Faceted Charts confirm that companies are not adopting skills in isolation. As Technical skill demand rises (the blue line, for example), the Soft skill demand (the orange line) either maintains its high level or slightly increases, creating the fertile ground for the identified hybrid roles. This pattern underscores a fundamental insight for career planning: future value lies not in mastering one category of skill, but in the fluent integration of technical expertise with the cognitive and interpersonal competencies necessary for collaborative problem-solving.

# Machine Learning with PySpark MLlib

#### Goal 3 - Measure skill overlap between job titles (similarity metrics)

- Quantify how closely related two job titles are based on shared skills.

In [ ]:
df_cleaned.printSchema()
df_pandas = df_cleaned

In [ ]:
# =============================================================================
# GOAL 3: Geographic Skill Similarity - Country Overlap (PySpark Native)
# This code calculates the Cosine Similarity between the aggregated skill sets
# of different countries using PySpark ML and SQL functions.
# =============================================================================
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list, concat_ws, size, udf, lit, explode, split, lower, when, trim
from pyspark.sql.types import ArrayType, StringType, DoubleType, MapType
from pyspark.ml.feature import CountVectorizer, IDF, Tokenizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt
from functools import reduce

print("=" * 70)
print("GOAL 3: GEOGRAPHIC SKILL SIMILARITY ANALYSIS (PySpark)")
print("=" * 70)

try:
    # --- Define Column Names ---
    GROUPING_COL = 'search_country'
    RAW_SKILL_COL = 'skills' # Use the raw string column from the provided schema
    VECTOR_COL = 'features'

    # 0. Preparation and Filtering (PySpark)
    if RAW_SKILL_COL not in df_cleaned.columns:
        raise KeyError(f"Required column '{RAW_SKILL_COL}' is missing in df_cleaned.")

    print(f"🔄 Preparing skills using Goal 4's robust tokenization (comma split + trim)...")

    # Calculate top countries for filtering (done before tokenization for simplicity)
    country_counts = df_cleaned.groupBy(GROUPING_COL).count()
    top_countries_list = [row[GROUPING_COL] for row in country_counts.orderBy(col("count").desc()).limit(15).collect()]

    df_filtered = df_cleaned.filter(col(GROUPING_COL).isin(top_countries_list))

    # 1. Aggregate Skills per Country (PySpark SQL)
    # Applying Goal 4's robust tokenization:

    # 1.1. Tokenize: Create an array of individual skills, splitting by comma (',')
    df_tokenized = df_filtered.withColumn(
        "skill_array",
        split(lower(col(RAW_SKILL_COL)), ",")
    )

    # 1.2. Explode the array into new rows (one skill per row)
    df_exploded_skills = df_tokenized.withColumn(
        "individual_skill_raw",
        explode(col("skill_array"))
    )

    # 1.3. Clean: Apply trim and filter out empty tokens, creating the final token column
    df_cleaned_skills = df_exploded_skills.withColumn(
        "skill_token",
        trim(col("individual_skill_raw"))
    ).filter(col("skill_token") != "") # Filter out empty tokens

    # 1.4. Group by country and collect all individual skill tokens into a single array of strings.
    df_country_skills = df_cleaned_skills.groupBy(GROUPING_COL).agg(
        collect_list("skill_token").alias("skill_list") # This is now correctly ARRAY<STRING>
    ).withColumn(
        # Convert list of skills to single space-separated string (for optional use/debugging)
        "skill_text",
        concat_ws(" ", col("skill_list"))
    )

    print(f"\n✅ Analyzing skill overlap across {len(top_countries_list)} countries using comma-separated tokens.")

    # 2. Feature Extraction (PySpark ML - TF/IDF Vectorization)
    # Use CountVectorizer to map skills to indices and get term frequency
    cv = CountVectorizer(inputCol="skill_list", outputCol="tf_vector", vocabSize=10000, minDF=2.0)
    cv_model = cv.fit(df_country_skills)
    df_vectorized = cv_model.transform(df_country_skills)

    # Create the IDF vector (optional but recommended for skill importance)
    idf = IDF(inputCol="tf_vector", outputCol=VECTOR_COL)
    idf_model = idf.fit(df_vectorized)
    df_vectorized = idf_model.transform(df_vectorized)

    # 3. Define Cosine Similarity UDF (PySpark UDF)

    # UDF to get vector components (non-zero indices/values)
    # We return the dict keys/values as strings for serialization, and the norm as a string.
    def get_vector_components(v):
        if v is None: return None, "0.0"
        # Convert Spark Vector to a dict {index: value}
        v_dict = {str(k): str(v) for k, v in zip(v.indices, v.values)}
        # Calculate L2 norm (magnitude)
        norm = sqrt(sum([val*val for val in v.values]))
        return v_dict, str(norm)

    # Use MapType for the dictionary component and StringType for the norm
    GetVectorComponentsUDF = udf(get_vector_components, MapType(StringType(), StringType()))

    df_vectors = df_vectorized.withColumn(
        "vector_data",
        GetVectorComponentsUDF(col(VECTOR_COL))
    ).select(
        col(GROUPING_COL).alias("country"),
        col("vector_data").getItem(0).alias("v_dict"),
        col("vector_data").getItem(1).alias("v_norm")
    ).cache()

    # UDF to calculate the dot product (Numerator of Cosine Similarity)
    def calculate_dot_product(dict1, dict2):
        if dict1 is None or dict2 is None: return 0.0

        # Determine the smaller dictionary for efficient iteration
        if len(dict1) > len(dict2):
            dict1, dict2 = dict2, dict1

        dot_product = 0.0
        # Keys are indices (strings), values are vector values (strings)
        for index_str, value1_str in dict1.items():
            value1 = float(value1_str)

            # Retrieve value from the other dictionary, defaulting to 0.0 if index is missing
            value2 = float(dict2.get(index_str, "0.0"))
            dot_product += value1 * value2

        return dot_product

    DotProductUDF = udf(calculate_dot_product, DoubleType())

    # 4. Calculate Similarity Matrix (PySpark Cross Join)
    # Cross join the data with itself to compare every country pair
    df_similarity = df_vectors.alias("A").crossJoin(df_vectors.alias("B")) \
        .filter(col("A.country") < col("B.country")) \
        .withColumn("dot_product", DotProductUDF(col("A.v_dict"), col("B.v_dict"))) \
        .withColumn("norm_product", col("A.v_norm").cast(DoubleType()) * col("B.v_norm").cast(DoubleType())) \
        .withColumn("Similarity",
                    when(col("norm_product") != 0, col("dot_product") / col("norm_product"))
                    .otherwise(lit(0.0))
        ) \
        .select(
            col("A.country").alias("Country 1"),
            col("B.country").alias("Country 2"),
            "Similarity"
        )

    similar_pairs_df = df_similarity.orderBy(col("Similarity").desc()).limit(10).toPandas()

    print("\n🏆 Top 10 Most Similar Country Pairs (Skill Overlap):")
    print(similar_pairs_df.to_string(index=False))

    # --- 5. Visualization: Similarity Heatmap (Requires full matrix) ---
    # Collect all pair-wise similarities
    full_similarity_data = df_similarity.union(
        df_similarity.select(col("Country 2").alias("Country 1"), col("Country 1").alias("Country 2"), col("Similarity"))
    ) \
    .union(
        df_vectors.select(col("country").alias("Country 1"), col("country").alias("Country 2"), lit(1.0).alias("Similarity"))
    )

    similarity_pd = full_similarity_data.toPandas()

    # Pivot the Pandas DataFrame to create the matrix format required for seaborn heatmap
    similarity_df = similarity_pd.pivot(index='Country 1', columns='Country 2', values='Similarity')
    similarity_df = similarity_df.fillna(1.0) # Fill diagonal (self-similarity) with 1

    # Order the columns/index consistently
    ordered_countries = similarity_df.mean(axis=1).sort_values(ascending=False).index.tolist()
    similarity_df = similarity_df.reindex(index=ordered_countries, columns=ordered_countries)


    plt.figure(figsize=(12, 10))
    sns.heatmap(similarity_df, annot=True, fmt='.2f', cmap='YlGnBu',
                square=True, linewidths=0.5, cbar_kws={'label': 'Cosine Similarity'})
    plt.title('Country Skill Similarity Matrix (Top 15 Countries)', fontsize=13, fontweight='bold')
    plt.xticks(rotation=45, ha='right', fontsize=9)
    plt.yticks(rotation=0, fontsize=9)
    plt.tight_layout()
    plt.show()

    print("\n✅ GOAL 3 COMPLETE (Country Similarity)\n")

except NameError as e:
    print(f"\nERROR: The PySpark DataFrame 'df_cleaned' is not defined: {e}")
    print("Please ensure your data loading and cleaning steps have been run to create 'df_cleaned'.")
except KeyError as e:
    print(f"\nKEY ERROR: A required column is missing: {e}")
    print("Please check the PySpark DataFrame 'df_cleaned' schema and ensure it contains 'search_country' and 'skills'.")
except Exception as e:
    print(f"\nAn unexpected error occurred during Goal 3 analysis: {e}")

#### Goal 5 - Evaluate emerging job clusters (unsupervised ML)

- Cluster job roles based on skill similarity.

In [ ]:
# =============================================================================
# GOAL 6: Skill Evolution and Hybridization
# Analysis of how skill categories (Soft, Technical, Business, etc.) change over
# time and the emergence of "Hybrid" roles requiring multiple skill types.
# =============================================================================
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, when, count, array_contains, sum as spark_sum, explode
from pyspark.sql.types import ArrayType, StringType
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

print("=" * 70)
print("GOAL 6: SKILL EVOLUTION & HYBRIDIZATION")
print("=" * 70)

# NOTE: This code assumes 'df_cleaned' (or a suitable PySpark DataFrame
# with 'skills', 'company', and 'first_seen' columns) is available.
# It also assumes skills have been tokenized/cleaned into a list of strings
# within the 'skills_cleaned' column for accurate array analysis.

# --- 0. Setup and Data Preparation (Assuming df_cleaned exists) ---
# Create a dummy array column if skills_cleaned is a string for demonstration
# If your 'skills_cleaned' is already an array, skip this part.
if 'skills_cleaned' not in df_cleaned.columns or df_cleaned.schema['skills_cleaned'].dataType != ArrayType(StringType()):
    # Replace with your actual cleaning/tokenization logic if needed
    print("WARNING: 'skills_cleaned' not found or not an ArrayType. Using raw 'skills' column.")
    df_with_array = df_cleaned.withColumn("skills_array", F.split(F.lower(col("skills")), " "))
else:
    df_with_array = df_cleaned.withColumnRenamed("skills_cleaned", "skills_array")

# Filter data and extract year
df_filtered = df_with_array.filter(col("skills_array").isNotNull())
df_with_year = df_filtered.withColumn("year", year(col("first_seen")))

# --- 1. Skill Classification Dictionaries (Simplified) ---
# NOTE: In a real scenario, these dictionaries would be much more extensive.
print("[Step 1] Defining skill categories...")

TECH_SKILLS = ["python", "java", "sql", "aws", "azure", "docker", "kubernetes", "react", "c++", "tableau"]
SOFT_SKILLS = ["communication", "leadership", "teamwork", "problem-solving", "collaboration", "adaptability", "mentoring"]
BUSINESS_SKILLS = ["finance", "budgeting", "strategy", "marketing", "sales", "p&l", "forecasting"]

# --- 2. Feature Engineering: Skill Category Flags ---
print("[Step 2] Labeling job postings with skill categories...")

# Function to check if any skill in a list is present in the job's skills_array
def check_category(skill_list):
    return array_contains(col("skills_array"), skill_list[0]) | \
           reduce(lambda a, b: a | b, [array_contains(col("skills_array"), skill.lower()) for skill in skill_list[1:]])

df_labeled = df_with_year \
    .withColumn("has_tech", check_category(TECH_SKILLS)) \
    .withColumn("has_soft", check_category(SOFT_SKILLS)) \
    .withColumn("has_business", check_category(BUSINESS_SKILLS))

# --- 3. Defining Hybrid and Primary Skill Categories ---
print("[Step 3] Defining Hybrid and Primary Skill Categories...")

df_categorized = df_labeled.withColumn("is_hybrid",
    when(col("has_tech") & col("has_soft"), True).otherwise(False)
)

# Identify the dominant skill category for non-hybrid jobs
df_categorized = df_categorized.withColumn("skill_category",
    when(col("has_tech"), "Technical")
    .when(col("has_soft"), "Soft")
    .when(col("has_business"), "Business")
    .otherwise("Other")
)

# --- 4. Evolution of Hybrid Roles (for Visualization 1 & 4) ---
print("[Step 4] Calculating Hybrid Skill Evolution...")
df_hybrid_evolution = df_categorized.groupBy("year").agg(
    spark_sum(when(col("is_hybrid"), 1).otherwise(0)).alias("hybrid_count"),
    count("*").alias("total_jobs")
).withColumn("hybrid_percentage", (col("hybrid_count") / col("total_jobs")) * 100)

# --- 5. Skill Category Normalization (for Visualization 2 & 3) ---
# Goal: Get the total count of each category (Tech, Soft, Business, Other) per year.
print("[Step 5] Calculating Category Evolution and preparing for Heatmap...")

# Explode the skills_array to count individual skill occurrences (for Visualization 3/Heatmap)
df_skill_counts = df_with_year.withColumn("skill", explode(col("skills_array")))
df_skill_counts = df_skill_counts.filter(col("skill") != "")

# Aggregate counts for the top 10 companies/industries
top_companies = df_cleaned.groupBy("company").count().orderBy(col("count").desc()).limit(10).toPandas()['company'].tolist()

# Aggregate categorized counts by year and company for Visualization 5 (Faceted Charts)
df_filtered_categories = df_categorized.filter(col("company").isin(top_companies))

# --- 6. Visualization Data Preparation ---
# Collect data for plotting (conversion to Pandas)
print("[Step 6] Normalizing data for visualization...")

# Aggregation for Stacked Area Chart (Viz 2)
df_category_evolution = df_categorized.groupBy("year").pivot("skill_category").agg(count("*")).fillna(0).toPandas()
df_category_evolution.set_index('year', inplace=True)
# Normalize to percentages
df_category_evolution = df_category_evolution.div(df_category_evolution.sum(axis=1), axis=0) * 100
df_category_evolution.sort_index(inplace=True)

# Latest Year Heatmap Data (Viz 3)
latest_year = df_categorized.select(spark_sum(col("year")).alias("total_year_sum")).first()["total_year_sum"]
if latest_year is not None:
    latest_year = df_categorized.select(col("year")).distinct().orderBy(col("year").desc()).limit(1).collect()[0]['year']
    df_latest = df_categorized.filter(col("year") == latest_year)

    df_viz_normalized = df_latest.groupBy("company").agg(
        spark_sum(when(col("skill_category") == "Technical", 1).otherwise(0)).alias("Technical"),
        spark_sum(when(col("skill_category") == "Soft", 1).otherwise(0)).alias("Soft"),
        spark_sum(when(col("skill_category") == "Business", 1).otherwise(0)).alias("Business"),
    ).filter(col("company").isin(top_companies)).toPandas()

    # Normalize the counts for heatmap (row-wise percentages)
    df_viz_normalized.set_index('company', inplace=True)
    df_viz_normalized = df_viz_normalized.div(df_viz_normalized.sum(axis=1), axis=0) * 100
    df_viz_normalized = df_viz_normalized[['Technical', 'Soft', 'Business']]
    print("✓ Data normalized")
else:
    print("WARNING: Cannot determine latest year for heatmap. Skipping Viz 3.")
    df_viz_normalized = pd.DataFrame()


# --- VISUALIZATION SECTION ---
print("\n[Visualization] Creating charts...")

# --- Get necessary Pandas DataFrames for plotting ---
hybrid_pd = df_hybrid_evolution.toPandas()
category_time_pd = df_filtered_categories.groupBy("year", "company", "skill_category").agg(
    count("job_link").alias("count")
).toPandas()
# Calculate percentages per company-year for Faceted Plot
category_time_pd['total'] = category_time_pd.groupby(['year', 'company'])['count'].transform('sum')
category_time_pd['percentage'] = (category_time_pd['count'] / category_time_pd['total']) * 100
category_time_pd = category_time_pd.dropna() # Drop any rows where total is 0

# --- VISUALIZATION 1: Hybrid Skill Evolution Line Chart ---
print("  → Creating Visualization 1: Hybrid Skill Evolution")
plt.figure(figsize=(10, 6))
sns.lineplot(x='year', y='hybrid_percentage', data=hybrid_pd, marker='o',
             linewidth=3, color='#4CAF50', label='Hybrid Roles')
plt.title('Percentage of Hybrid (Tech + Soft) Roles Over Time', fontsize=14, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Percentage of Total Jobs (%)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(hybrid_pd['year'].unique(), rotation=45)
plt.tight_layout()
plt.savefig('goal_6_hybrid_evolution.png', dpi=300)
plt.show()
print("    ✓ Saved: goal_6_hybrid_evolution.png")


# --- VISUALIZATION 2: Stacked Area Chart - Category Evolution ---
print("\n  → Creating Visualization 2: Skill Category Evolution")
plt.figure(figsize=(12, 7))
df_category_evolution.plot(kind='area', stacked=True, alpha=0.8,
                           color={'Technical': '#007BFF', 'Soft': '#FFC107', 'Business': '#28A745', 'Other': '#6C757D'},
                           ax=plt.gca())
plt.title('Proportional Evolution of Primary Skill Categories', fontsize=14, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Proportion of Total Jobs (%)', fontsize=12)
plt.legend(title='Category', loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('goal_6_category_evolution_stacked.png', dpi=300)
plt.show()
print("    ✓ Saved: goal_6_category_evolution_stacked.png")


# --- VISUALIZATION 3: Heatmap - Latest Year Category Distribution ---
if not df_viz_normalized.empty:
    print("\n  → Creating Visualization 3: Latest Year Category Distribution Heatmap")
    plt.figure(figsize=(10, 8))
    sns.heatmap(df_viz_normalized, annot=True, fmt=".1f", cmap="Blues",
                linewidths=.5, cbar_kws={'label': 'Proportion of Jobs (%)'})
    plt.title(f'Skill Category Distribution by Company ({latest_year})', fontsize=14, fontweight='bold')
    plt.ylabel('Company', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('goal_6_category_heatmap.png', dpi=300)
    plt.show()
    print("    ✓ Saved: goal_6_category_heatmap.png")


# --- VISUALIZATION 4: Hybrid vs Non-Hybrid Comparison (Bar Chart) ---
print("\n  → Creating Visualization 4: Hybrid vs Non-Hybrid Job Counts")
hybrid_counts = df_categorized.groupBy("is_hybrid").count().toPandas()
plt.figure(figsize=(8, 5))
sns.barplot(x='is_hybrid', y='count', data=hybrid_counts, palette=['#FF5733', '#4CAF50'])
plt.title('Total Job Postings: Hybrid vs. Non-Hybrid', fontsize=14, fontweight='bold')
plt.xlabel('Job Type (False=Non-Hybrid, True=Hybrid)', fontsize=12)
plt.ylabel('Total Count of Job Postings', fontsize=12)
plt.xticks(ticks=[0, 1], labels=['Non-Hybrid', 'Hybrid'])
plt.tight_layout()
plt.savefig('goal_6_hybrid_comparison.png', dpi=300)
plt.show()
print("    ✓ Saved: goal_6_hybrid_comparison.png")


# --- VISUALIZATION 5: Faceted Line Charts by Company ---
if not category_time_pd.empty and not top_companies.empty:
    print("\n  → Creating Visualization 5: Category Evolution by Company (Faceted)")

    # Filter for the main categories for clarity in facets
    category_time_pd_filtered = category_time_pd[category_time_pd['skill_category'].isin(['Technical', 'Soft', 'Business'])]

    num_companies = len(top_companies)
    if num_companies > 0:
        rows = int(np.ceil(num_companies / 5))
        cols = min(num_companies, 5)

        fig, axes = plt.subplots(rows, cols, figsize=(20, 4 * rows), sharey=True)
        # Ensure axes is an iterable array even if only one subplot exists
        axes = axes.flatten() if isinstance(axes, np.ndarray) else [axes]

        for idx, company in enumerate(top_companies):
            if idx >= len(axes): continue
            ax = axes[idx]
            company_data = category_time_pd_filtered[category_time_pd_filtered['company'] == company]

            # Iterate over unique categories in the company_data DataFrame (FIXED)
            sns.lineplot(x='year', y='percentage', hue='skill_category', data=company_data,
                         marker='o', ax=ax, linewidth=2)

            ax.set_title(company, fontsize=11, fontweight='bold')
            ax.set_xlabel('Year', fontsize=9)
            ax.set_ylabel('Percentage (%)', fontsize=9)
            ax.grid(True, alpha=0.3)

            # Only add legend to the first plot to avoid clutter
            if idx == 0:
                ax.legend(fontsize=7, loc='best', title='Category')
            else:
                ax.get_legend().remove()

        # Hide any unused subplots
        for i in range(num_companies, len(axes)):
            fig.delaxes(axes[i])

        plt.suptitle('Skill Category Evolution by Company Over Time',
                     fontsize=16, fontweight='bold', y=1.02)
        plt.tight_layout(rect=[0, 0, 1, 0.98]) # Adjust layout to make space for suptitle
        plt.savefig('goal_6_category_evolution_faceted.png', dpi=300, bbox_inches='tight')
        plt.show()
        print("    ✓ Saved: goal_6_category_evolution_faceted.png")
    else:
         print("WARNING: Not enough top companies found for faceted visualization.")
else:
    print("WARNING: DataFrames for Faceted Line Charts are empty. Skipping Viz 5.")


print("\n✅ GOAL 6 EXECUTION COMPLETE\n")

Save Results

In [ ]:
# =============================================================================
# SUMMARY
# =============================================================================
print("=" * 70)
print("🎯 ALL GOALS COMPLETED")
print("=" * 70)

# =============================================================================
# CLEANUP: Stop Spark Session
# =============================================================================
print("\n🔧 Cleaning up resources...")

try:
    if 'spark' in locals():
        spark.stop()
        print("✅ Spark session stopped successfully")
    else:
        print("⚠️  No active Spark session found")
except Exception as e:
    print(f"⚠️  Error stopping Spark session: {e}")

print("\n" + "=" * 70)
print("✅ ALL PROCESSING COMPLETE - SESSION CLOSED")
print("=" * 70)